# * VINSIGHT : CEO Report

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100GEO' --Total Revenue (Geo)
        , 'B1R000100' --Prepaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'TB3S000600' --FTTx Reported SubBase
        , 'TB4S000500' --TVS Active Subs
         
        , 'B1S000101CS' --Prepaid Gross Adds
        , 'B2S010100CS' --Postpaid Gross Adds B2C
        , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
        , 'TB4S001400CS' --TVS Now Gross Adds
         
        , 'B1S000200' --Prepaid Churn Subs
        , 'B2S010200' --Postpaid Churn Subs B2C
        , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
        , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
        , 'TSER13100' --FTTx Churn Subs
        , 'TSER13102' --FTTx Churn Subs Voluntary
        , 'TSER13103' --FTTx Churn Subs Involuntary
        , 'TSER13104' --FTTx Churn Subs Involuntary (Collection Forecast)
        , 'TSER14100' --TVS Churn Subs
        , 'TSER14102' --TVS Churn Subs Voluntary
        , 'TSER14103' --TVS Churn Subs Involuntary
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-07-03, 09:05:51

DataFrame: 12627 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

In [4]:
# chk_src_df.tail()

## Period Parameter

In [5]:
''' Parameter '''

# v_tm_key_qtr = curr_qtr
v_tm_key_qtr = 20241

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

# v_tm_key_wk = curr_wk
v_tm_key_wk = last_3_wk
# v_tm_key_wk = 2025020

print(f'v_tm_key_qtr: {v_tm_key_qtr}')
print(f'v_tm_key_mth: {v_tm_key_mth}')
print(f'v_tm_key_wk: {v_tm_key_wk}')

v_tm_key_qtr: 20241
v_tm_key_mth: 202506
v_tm_key_wk: 2025024


## Revenue(Geo) by Period

### Yearly

In [6]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df[col] = rev_geo_yearly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df

,TM_KEY_YR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2024,2025-07-03 03:13:24,"142,125,302,342","48,319,930,436","70,959,878,334","18,827,249,425","4,018,244,147"
1,2025,2025-07-03 03:13:24,"71,271,683,978","24,728,381,224","34,794,144,526","9,644,065,894","1,666,102,466"


### Quarterly

In [7]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df[col] = rev_geo_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df

,TM_KEY_QTR,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,20241,2025-07-03 03:13:24,"35,361,933,475","11,842,176,786","17,936,624,018","4,569,920,004","1,013,212,668"
1,20242,2025-07-03 03:13:24,"35,284,532,532","11,858,030,293","17,792,137,434","4,665,781,597","968,583,208"
2,20243,2025-07-03 03:13:24,"35,706,816,037","12,164,985,004","17,628,033,120","4,803,785,483","1,110,012,430"
3,20244,2025-07-03 03:13:24,"35,772,020,298","12,454,738,354","17,603,083,763","4,787,762,341","926,435,841"
4,20251,2025-07-03 03:13:24,"35,649,443,458","12,375,356,967","17,580,761,024","4,810,187,607","883,137,861"
5,20252,2025-07-03 03:13:24,"35,454,582,700","12,353,024,257","17,213,383,501","4,833,878,287","782,964,605"
6,20253,2025-07-03 03:13:24,"167,657,819",0,0,0,0


### Monthly

In [8]:
''' Revenue(Geo) Monthly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df[col] = rev_geo_monthly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df

,TM_KEY_MTH,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202506,2025-07-03 03:13:24,"11,544,526,903","4,019,648,792","5,458,620,339","1,606,925,707","188,000,015"
1,202507,2025-07-03 03:13:24,"167,657,819",0,0,0,0


### Weekly

In [9]:
''' Revenue(Geo) Weekly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]
# rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]
rev_geo_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = rev_geo_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df[col] = rev_geo_weekly_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,2025022,20250601,2025-07-03 03:13:24,"169,983,041","165,152,372","4,830,669",0,0
1,2025023,20250608,2025-07-03 03:13:24,"2,756,838,432","993,554,639","1,356,244,391","366,070,732","40,968,670"
2,2025024,20250615,2025-07-03 03:13:24,"2,695,922,607","871,828,576","1,373,464,775","403,856,605","46,772,652"
3,2025025,20250622,2025-07-03 03:13:24,"3,022,777,187","877,080,187","1,584,885,025","501,698,503","59,113,473"
4,2025026,20250629,2025-07-03 03:13:24,"2,698,942,686","919,132,694","1,139,195,479","335,299,868","38,795,687"
5,2025027,20250702,2025-07-03 03:13:24,"367,720,769","192,900,324",0,0,"2,349,533"


### Daily
    The information is valid until 28 June 2025

In [10]:
''' Revenue(Geo) Daily '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'B2R010100' #Postpaid Revenue B2C
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'POST_B2C':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'TOTAL(Geo)', 'PRE', 'POST_B2C', 'TOL', 'TVS']]

mod_col_list = rev_geo_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df[col] = rev_geo_daily_df[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,TOTAL(Geo),PRE,POST_B2C,TOL,TVS
0,202507,20250702,2025-07-03 03:13:24,0,0,0,0,0
1,202507,20250701,2025-07-03 03:13:24,"167,657,819",0,0,0,0
2,202506,20250630,2025-07-03 03:13:24,"200,062,950","192,900,324",0,0,"2,349,533"
3,202506,20250629,2025-07-03 03:13:24,"402,909,612","136,390,655",0,0,0
4,202506,20250628,2025-07-03 03:13:24,"740,535,388","135,835,796","413,816,520","170,863,107","20,019,964"
5,202506,20250627,2025-07-03 03:13:24,"224,383,336","136,543,291","87,840,045",0,0
6,202506,20250626,2025-07-03 03:13:24,"218,118,737","130,990,811","87,127,926",0,0
7,202506,20250625,2025-07-03 03:13:24,"713,502,930","134,721,573","395,568,873","164,436,761","18,775,723"
8,202506,20250624,2025-07-03 03:13:24,"197,907,998","120,455,470","77,452,528",0,0
9,202506,20250623,2025-07-03 03:13:24,"201,584,684","124,195,096","77,389,588",0,0


## Subscriber by Period

### Yearly

In [11]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['FTTX_RPT'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df[col] = sub_yearly_df[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2024,2025-07-03 03:13:24,"28,100,141","14,158,698","3,154,889","1,180,410"
1,2025,2025-07-03 03:13:24,"26,270,901","14,031,799","3,207,941","1,112,132"


### Quarterly

In [12]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['FTTX_RPT'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df[col] = sub_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,20241,2025-07-03 03:13:24,0,0,"3,090,287","1,258,065"
1,20242,2025-07-03 03:13:24,"27,800,517","14,226,167","3,107,084","1,236,606"
2,20243,2025-07-03 03:13:24,"27,505,003","14,186,615","3,127,304","1,205,116"
3,20244,2025-07-03 03:13:24,"28,100,141","14,158,698","3,154,889","1,180,410"
4,20251,2025-07-03 03:13:24,"27,140,083","14,084,812","3,187,717","1,153,137"
5,20252,2025-07-03 03:13:24,"26,257,033","14,031,799","3,207,941","1,112,829"
6,20253,2025-07-03 03:13:24,"26,270,901",0,0,"1,112,132"


### Monthly

In [13]:
''' Subscriber Monthly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['FTTX_RPT'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df[col] = sub_monthly_df[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202506,2025-07-03 03:13:24,"26,257,033","14,031,799","3,207,941","1,112,829"
1,202507,2025-07-03 03:13:24,"26,270,901",0,0,"1,112,132"


### Weekly

In [14]:
''' Subscriber Weekly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]
# sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['FTTX_RPT'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]
sub_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = sub_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df[col] = sub_weekly_df[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,2025022,20250601,2025-07-03 03:13:24,"26,438,734","14,073,719","3,205,472","1,128,753"
1,2025023,20250608,2025-07-03 03:13:24,"26,362,107","14,064,757","3,202,722","1,126,877"
2,2025024,20250615,2025-07-03 03:13:24,"26,318,308","14,045,663","3,201,297","1,119,850"
3,2025025,20250622,2025-07-03 03:13:24,"26,199,621","14,044,858","3,204,117","1,117,130"
4,2025026,20250629,2025-07-03 03:13:24,"26,257,033","14,031,799","3,204,018","1,113,416"
5,2025027,20250701,2025-07-03 03:13:24,"26,270,901",0,"3,207,941","1,112,132"


### Daily

In [15]:
''' Subscriber Daily '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB3S000600' #FTTx Reported SubBase
    , 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]
sub_daily_df = sub_daily_df.loc[sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['FTTX_RPT'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'POST_RPT_B2C':'sum', 'FTTX_RPT':'sum', 'TVS_ACTV':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'POST_RPT_B2C', 'FTTX_RPT', 'TVS_ACTV']]

mod_col_list = sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df[col] = sub_daily_df[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,POST_RPT_B2C,FTTX_RPT,TVS_ACTV
0,202507,20250702,2025-07-03 03:13:24,0,0,0,0
1,202507,20250701,2025-07-03 03:13:24,"26,270,901",0,0,"1,112,132"
2,202506,20250630,2025-07-03 03:13:24,"9,900,021",0,"3,207,941","1,112,829"
3,202506,20250629,2025-07-03 03:13:24,"26,257,033","14,031,799","3,204,018","1,113,416"
4,202506,20250628,2025-07-03 03:13:24,"26,259,869","14,035,407","3,202,559","1,114,091"
5,202506,20250627,2025-07-03 03:13:24,"26,248,248","14,040,552","3,206,747","1,114,091"
6,202506,20250626,2025-07-03 03:13:24,"26,228,435","14,037,713","3,205,126","1,114,199"
7,202506,20250625,2025-07-03 03:13:24,"26,224,155","14,032,939","3,201,695","1,115,415"
8,202506,20250624,2025-07-03 03:13:24,"26,202,629","14,049,076","3,207,278","1,115,883"
9,202506,20250623,2025-07-03 03:13:24,"26,213,609","14,047,730","3,206,076","1,116,503"


## Gross Adds by Period

### Yearly

In [16]:
''' Gross Adds Yearly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_yearly_df = ga_yearly_df.loc[ga_yearly_df['METRIC_CD'].isin(v_metric_list)]

ga_yearly_df['PRE_GA'] = np.where(ga_yearly_df['METRIC_CD']=='B1S000101CS', ga_yearly_df['P'], 0)
ga_yearly_df['POST_GA_B2C'] = np.where(ga_yearly_df['METRIC_CD']=='B2S010100CS', ga_yearly_df['P'], 0)
ga_yearly_df['TOL_GA_CONN_CON'] = np.where(ga_yearly_df['METRIC_CD']=='TB3S000102CS', ga_yearly_df['P'], 0)
ga_yearly_df['TVS_NOW_GA'] = np.where(ga_yearly_df['METRIC_CD']=='TB4S001400CS', ga_yearly_df['P'], 0)

ga_yearly_df = ga_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_yearly_df = ga_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
ga_yearly_df = ga_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_yearly_df[col] = ga_yearly_df[col].apply(lambda x: format(x, ',.0f'))
ga_yearly_df

,TM_KEY_YR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2024,2025-07-03 03:13:24,"30,607,404","2,642,885","558,527","7,682"
1,2025,2025-07-03 03:13:24,"12,175,482","1,119,610","298,322","9,635"


### Quarterly

In [17]:
''' Gross Adds Quarterly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['METRIC_CD'].isin(v_metric_list)]
ga_quarterly_df = ga_quarterly_df.loc[ga_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

ga_quarterly_df['PRE_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='B1S000101CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['POST_GA_B2C'] = np.where(ga_quarterly_df['METRIC_CD']=='B2S010100CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TOL_GA_CONN_CON'] = np.where(ga_quarterly_df['METRIC_CD']=='TB3S000102CS', ga_quarterly_df['P'], 0)
ga_quarterly_df['TVS_NOW_GA'] = np.where(ga_quarterly_df['METRIC_CD']=='TB4S001400CS', ga_quarterly_df['P'], 0)

ga_quarterly_df = ga_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_quarterly_df = ga_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
ga_quarterly_df = ga_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_quarterly_df[col] = ga_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
ga_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,20241,2025-07-03 03:13:24,"8,953,743","701,773","130,958",0
1,20242,2025-07-03 03:13:24,"7,514,316","698,111","141,474",0
2,20243,2025-07-03 03:13:24,"7,156,127","662,059","147,285","4,992"
3,20244,2025-07-03 03:13:24,"6,983,218","580,942","138,810","2,690"
4,20251,2025-07-03 03:13:24,"6,673,218","569,340","145,692","3,337"
5,20252,2025-07-03 03:13:24,"5,502,264","550,270","150,911","6,298"
6,20253,2025-07-03 03:13:24,0,0,"1,719",0


### Monthly

In [18]:
''' Gross Adds Monthly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['METRIC_CD'].isin(v_metric_list)]
ga_monthly_df = ga_monthly_df.loc[ga_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_monthly_df['PRE_GA'] = np.where(ga_monthly_df['METRIC_CD']=='B1S000101CS', ga_monthly_df['P'], 0)
ga_monthly_df['POST_GA_B2C'] = np.where(ga_monthly_df['METRIC_CD']=='B2S010100CS', ga_monthly_df['P'], 0)
ga_monthly_df['TOL_GA_CONN_CON'] = np.where(ga_monthly_df['METRIC_CD']=='TB3S000102CS', ga_monthly_df['P'], 0)
ga_monthly_df['TVS_NOW_GA'] = np.where(ga_monthly_df['METRIC_CD']=='TB4S001400CS', ga_monthly_df['P'], 0)

ga_monthly_df = ga_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_monthly_df = ga_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
ga_monthly_df = ga_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    ga_monthly_df[col] = ga_monthly_df[col].apply(lambda x: format(x, ',.0f'))
ga_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202506,2025-07-03 03:13:24,"1,710,123","170,277","49,688","1,740"
1,202507,2025-07-03 03:13:24,0,0,"1,719",0


### Weekly

In [19]:
''' Gross Adds Weekly '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['METRIC_CD'].isin(v_metric_list)]
# ga_weekly_df = ga_weekly_df.loc[ga_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

ga_weekly_df['PRE_GA'] = np.where(ga_weekly_df['METRIC_CD']=='B1S000101CS', ga_weekly_df['P'], 0)
ga_weekly_df['POST_GA_B2C'] = np.where(ga_weekly_df['METRIC_CD']=='B2S010100CS', ga_weekly_df['P'], 0)
ga_weekly_df['TOL_GA_CONN_CON'] = np.where(ga_weekly_df['METRIC_CD']=='TB3S000102CS', ga_weekly_df['P'], 0)
ga_weekly_df['TVS_NOW_GA'] = np.where(ga_weekly_df['METRIC_CD']=='TB4S001400CS', ga_weekly_df['P'], 0)

ga_weekly_df = ga_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_weekly_df = ga_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
ga_weekly_df = ga_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]
ga_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = ga_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_weekly_df[col] = ga_weekly_df[col].apply(lambda x: format(x, ',.0f'))
ga_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,2025022,20250601,2025-07-03 03:13:24,"64,643","5,571","1,670",69
1,2025023,20250608,2025-07-03 03:13:24,"421,599","42,351","12,378",341
2,2025024,20250615,2025-07-03 03:13:24,"387,136","40,541","11,591",281
3,2025025,20250622,2025-07-03 03:13:24,"396,623","39,736","11,969",358
4,2025026,20250629,2025-07-03 03:13:24,"440,122","36,114","12,080",508
5,2025027,20250702,2025-07-03 03:13:24,0,"5,964","1,719",183


### Daily

In [20]:
''' Gross Adds Daily '''

v_metric_list = [
    'B1S000101CS' #Prepaid Gross Adds
    , 'B2S010100CS' #Postpaid Gross Adds B2C
    , 'TB3S000102CS' #TOL Gross Adds Connected : Consumer
    , 'TB4S001400CS' #TVS Now Gross Adds
    ]

ga_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
ga_daily_df = ga_daily_df.loc[ga_daily_df['METRIC_CD'].isin(v_metric_list)]
ga_daily_df = ga_daily_df.loc[ga_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

ga_daily_df['PRE_GA'] = np.where(ga_daily_df['METRIC_CD']=='B1S000101CS', ga_daily_df['P'], 0)
ga_daily_df['POST_GA_B2C'] = np.where(ga_daily_df['METRIC_CD']=='B2S010100CS', ga_daily_df['P'], 0)
ga_daily_df['TOL_GA_CONN_CON'] = np.where(ga_daily_df['METRIC_CD']=='TB3S000102CS', ga_daily_df['P'], 0)
ga_daily_df['TVS_NOW_GA'] = np.where(ga_daily_df['METRIC_CD']=='TB4S001400CS', ga_daily_df['P'], 0)

ga_daily_df = ga_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_GA':'sum', 'POST_GA_B2C':'sum', 'TOL_GA_CONN_CON':'sum', 'TVS_NOW_GA':'sum'})
ga_daily_df = ga_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
ga_daily_df = ga_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_GA', 'POST_GA_B2C', 'TOL_GA_CONN_CON', 'TVS_NOW_GA']]

mod_col_list = ga_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    ga_daily_df[col] = ga_daily_df[col].apply(lambda x: format(x, ',.0f'))
ga_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_GA,POST_GA_B2C,TOL_GA_CONN_CON,TVS_NOW_GA
0,202507,20250702,2025-07-03 03:13:24,0,0,0,0
1,202507,20250701,2025-07-03 03:13:24,0,0,"1,719",0
2,202506,20250630,2025-07-03 03:13:24,0,"5,964",0,183
3,202506,20250629,2025-07-03 03:13:24,"61,926","5,231","1,816",97
4,202506,20250628,2025-07-03 03:13:24,"66,513","6,151","2,001",91
5,202506,20250627,2025-07-03 03:13:24,"73,911","4,710","1,864",73
6,202506,20250626,2025-07-03 03:13:24,"67,875","4,481","1,634",57
7,202506,20250625,2025-07-03 03:13:24,"58,617","4,806","1,567",66
8,202506,20250624,2025-07-03 03:13:24,"56,195","5,742","1,581",63
9,202506,20250623,2025-07-03 03:13:24,"55,085","4,993","1,617",61


## Churn Subs by Period

### Yearly

In [21]:
''' Churn Subs Yearly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_yearly_df = churn_yearly_df.loc[churn_yearly_df['METRIC_CD'].isin(v_metric_list)]

churn_yearly_df['PRE_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='B1S000200', churn_yearly_df['P'], 0)
churn_yearly_df['POST_CHURN_B2C'] = np.where(churn_yearly_df['METRIC_CD']=='B2S010200', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13100', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_VOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13102', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13103', churn_yearly_df['P'], 0)
churn_yearly_df['FTTX_INVOLUN(CF)'] = np.where(churn_yearly_df['METRIC_CD']=='TSER13104', churn_yearly_df['P'], 0)
churn_yearly_df['TVS_CHURN'] = np.where(churn_yearly_df['METRIC_CD']=='TSER14100', churn_yearly_df['P'], 0)

churn_yearly_df = churn_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_yearly_df = churn_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
churn_yearly_df = churn_yearly_df[['TM_KEY_YR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_yearly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_yearly_df[col] = churn_yearly_df[col].apply(lambda x: format(x, ',.0f'))
churn_yearly_df

,TM_KEY_YR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2024,2025-07-03 03:13:24,0,"2,857,216","456,668","170,154","281,222","282,188","194,250"
1,2025,2025-07-03 03:13:24,0,"1,140,067","213,590","80,888","139,087","174,030","114,534"


### Quarterly

In [22]:
''' Churn Subs Quarterly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['METRIC_CD'].isin(v_metric_list)]
churn_quarterly_df = churn_quarterly_df.loc[churn_quarterly_df['TM_KEY_QTR']>=v_tm_key_qtr]

churn_quarterly_df['PRE_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='B1S000200', churn_quarterly_df['P'], 0)
churn_quarterly_df['POST_CHURN_B2C'] = np.where(churn_quarterly_df['METRIC_CD']=='B2S010200', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13100', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_VOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13102', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13103', churn_quarterly_df['P'], 0)
churn_quarterly_df['FTTX_INVOLUN(CF)'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER13104', churn_quarterly_df['P'], 0)
churn_quarterly_df['TVS_CHURN'] = np.where(churn_quarterly_df['METRIC_CD']=='TSER14100', churn_quarterly_df['P'], 0)

churn_quarterly_df = churn_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_quarterly_df = churn_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
churn_quarterly_df = churn_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_quarterly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_quarterly_df[col] = churn_quarterly_df[col].apply(lambda x: format(x, ',.0f'))
churn_quarterly_df

,TM_KEY_QTR,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,20241,2025-07-03 03:13:24,0,"871,395","118,742","43,223","72,945","73,317","48,913"
1,20242,2025-07-03 03:13:24,0,"718,051","114,261","43,677","70,807","70,972","40,970"
2,20243,2025-07-03 03:13:24,0,"664,232","118,872","44,269","67,909","68,985","48,795"
3,20244,2025-07-03 03:13:24,0,"603,538","104,793","38,985","69,561","68,914","55,572"
4,20251,2025-07-03 03:13:24,0,"553,494","104,815","38,083","68,600","68,746","47,799"
5,20252,2025-07-03 03:13:24,0,"586,573","108,775","42,805","70,487","105,284","66,735"
6,20253,2025-07-03 03:13:24,0,0,0,0,0,0,0


### Monthly

In [23]:
''' Churn Subs Monthly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['METRIC_CD'].isin(v_metric_list)]
churn_monthly_df = churn_monthly_df.loc[churn_monthly_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_monthly_df['PRE_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='B1S000200', churn_monthly_df['P'], 0)
churn_monthly_df['POST_CHURN_B2C'] = np.where(churn_monthly_df['METRIC_CD']=='B2S010200', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13100', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_VOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13102', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13103', churn_monthly_df['P'], 0)
churn_monthly_df['FTTX_INVOLUN(CF)'] = np.where(churn_monthly_df['METRIC_CD']=='TSER13104', churn_monthly_df['P'], 0)
churn_monthly_df['TVS_CHURN'] = np.where(churn_monthly_df['METRIC_CD']=='TSER14100', churn_monthly_df['P'], 0)

churn_monthly_df = churn_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_monthly_df = churn_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
churn_monthly_df = churn_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_monthly_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    churn_monthly_df[col] = churn_monthly_df[col].apply(lambda x: format(x, ',.0f'))
churn_monthly_df

,TM_KEY_MTH,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202506,2025-07-03 03:13:24,0,"200,692","45,997","15,118","23,804","24,435","27,079"
1,202507,2025-07-03 03:13:24,0,0,0,0,0,0,0


### Weekly

In [24]:
''' Churn Subs Weekly '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_weekly_df = chk_src_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['METRIC_CD'].isin(v_metric_list)]
# churn_weekly_df = churn_weekly_df.loc[churn_weekly_df['TM_KEY_WK']>=v_tm_key_wk]

churn_weekly_df['PRE_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='B1S000200', churn_weekly_df['P'], 0)
churn_weekly_df['POST_CHURN_B2C'] = np.where(churn_weekly_df['METRIC_CD']=='B2S010200', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13100', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_VOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13102', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13103', churn_weekly_df['P'], 0)
churn_weekly_df['FTTX_INVOLUN(CF)'] = np.where(churn_weekly_df['METRIC_CD']=='TSER13104', churn_weekly_df['P'], 0)
churn_weekly_df['TVS_CHURN'] = np.where(churn_weekly_df['METRIC_CD']=='TSER14100', churn_weekly_df['P'], 0)

churn_weekly_df = churn_weekly_df.groupby('TM_KEY_WK').agg({'TM_KEY_DAY':'max', 'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_weekly_df = churn_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
churn_weekly_df = churn_weekly_df[['TM_KEY_WK', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]
churn_weekly_df.rename(columns={'TM_KEY_DAY':'AS_DAY'}, inplace=True)

mod_col_list = churn_weekly_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_weekly_df[col] = churn_weekly_df[col].apply(lambda x: format(x, ',.0f'))
churn_weekly_df

,TM_KEY_WK,AS_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,2025022,20250601,2025-07-03 03:13:24,0,"4,299",-814,641,0,"24,435","1,142"
1,2025023,20250608,2025-07-03 03:13:24,0,"49,654","14,695","3,956","1,905",0,"6,653"
2,2025024,20250615,2025-07-03 03:13:24,0,"48,846","12,507","3,525","8,490",0,"5,858"
3,2025025,20250622,2025-07-03 03:13:24,0,"41,706","8,670","3,444","4,097",0,"5,560"
4,2025026,20250629,2025-07-03 03:13:24,0,"51,597","10,939","3,552","9,312",0,"6,646"
5,2025027,20250702,2025-07-03 03:13:24,0,"4,590",0,0,0,0,"1,220"


### Daily

In [25]:
''' Churn Subs Daily '''

v_metric_list = [
    'B1S000200' #Prepaid Churn Subs
    , 'B2S010200' #Postpaid Churn Subs B2C
    # , 'B2S010201' #Postpaid Churn Subs Voluntary B2C
    # , 'B2S010202' #Postpaid Churn Subs Involuntary B2C
    , 'TSER13100' #FTTx Churn Subs
    , 'TSER13102' #FTTx Churn Subs Voluntary
    , 'TSER13103' #FTTx Churn Subs Involuntary
    , 'TSER13104' #FTTx Churn Subs Involuntary (Collection Forecast)
    , 'TSER14100' #TVS Churn Subs
    # , 'TSER14102' #TVS Churn Subs Voluntary
    # , 'TSER14103' #TVS Churn Subs Involuntary
    ]

churn_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
churn_daily_df = churn_daily_df.loc[churn_daily_df['METRIC_CD'].isin(v_metric_list)]
churn_daily_df = churn_daily_df.loc[churn_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

churn_daily_df['PRE_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='B1S000200', churn_daily_df['P'], 0)
churn_daily_df['POST_CHURN_B2C'] = np.where(churn_daily_df['METRIC_CD']=='B2S010200', churn_daily_df['P'], 0)
churn_daily_df['FTTX_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13100', churn_daily_df['P'], 0)
churn_daily_df['FTTX_VOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13102', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN'] = np.where(churn_daily_df['METRIC_CD']=='TSER13103', churn_daily_df['P'], 0)
churn_daily_df['FTTX_INVOLUN(CF)'] = np.where(churn_daily_df['METRIC_CD']=='TSER13104', churn_daily_df['P'], 0)
churn_daily_df['TVS_CHURN'] = np.where(churn_daily_df['METRIC_CD']=='TSER14100', churn_daily_df['P'], 0)

churn_daily_df = churn_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CHURN':'sum', 'POST_CHURN_B2C':'sum', 'FTTX_CHURN':'sum', 'FTTX_VOLUN':'sum', 'FTTX_INVOLUN':'sum', 'FTTX_INVOLUN(CF)':'sum', 'TVS_CHURN':'sum'})
churn_daily_df = churn_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
churn_daily_df = churn_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CHURN', 'POST_CHURN_B2C', 'FTTX_CHURN', 'FTTX_VOLUN', 'FTTX_INVOLUN', 'FTTX_INVOLUN(CF)', 'TVS_CHURN']]

mod_col_list = churn_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    churn_daily_df[col] = churn_daily_df[col].apply(lambda x: format(x, ',.0f'))
churn_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CHURN,POST_CHURN_B2C,FTTX_CHURN,FTTX_VOLUN,FTTX_INVOLUN,FTTX_INVOLUN(CF),TVS_CHURN
0,202507,20250702,2025-07-03 03:13:24,0,0,0,0,0,0,0
1,202507,20250701,2025-07-03 03:13:24,0,0,0,0,0,0,0
2,202506,20250630,2025-07-03 03:13:24,0,"4,590",0,0,0,0,"1,220"
3,202506,20250629,2025-07-03 03:13:24,0,"13,386",97,577,"3,059",0,"1,262"
4,202506,20250628,2025-07-03 03:13:24,0,566,"5,857",581,0,0,961
5,202506,20250627,2025-07-03 03:13:24,0,"7,770",61,536,"3,186",0,"1,070"
6,202506,20250626,2025-07-03 03:13:24,0,"3,735","-2,058",538,0,0,"1,003"
7,202506,20250625,2025-07-03 03:13:24,0,"12,037","7,067",306,1,0,516
8,202506,20250624,2025-07-03 03:13:24,0,"3,966",283,566,0,0,939
9,202506,20250623,2025-07-03 03:13:24,0,"10,137",-368,448,"3,066",0,895
